In [ ]:
import math
import random

import drawsvg as draw
from drawsvg import Drawing
from hyperbolic import euclid, util
from hyperbolic.poincare import *

from poincare_plane import get_poly_xy, Node

In [ ]:
from poincare_plane import hyp_poly_edge_construct

p = 5
corner_deg = 90
q = 360 / corner_deg
skip = 1.0
r = hyp_poly_edge_construct(p, q)
p_list = [
    Point.from_polar_euclid(r, deg=-skip*i*360/p)
    for i in range(p)
]
print(p_list)

n = len(p_list)
e_list = [Line.from_points(*p_list[i], *p_list[(i+1)%n]) for i in range(n)]
poly = Polygon(e_list, join=True)

d = Drawing(2.1, 2.1, origin='center')
d.draw(euclid.Circle(0, 0, 1), fill='silver')
for edge in e_list:
    d.draw(edge, hwidth=0.05, fill='blue')
# d.draw(poly, hwidth=(0,-0.15), fill='green')
# d.draw(poly, fill='black', opacity=0.3)
# for p in p_list:
#     d.draw(p, radius=0.05, stroke_width=0.01, stroke='#ccccff',
#               fill='none', opacity=0.6)

d.set_render_size(w=400)
d.save_svg('images/polyRegular.svg')
d

In [ ]:
from poincare_plane import construct_poly_vertices

def draw_poly(d, poly, color="#ffffff"):
    d.draw(poly, fill=color, opacity=0.25)
    d.draw(poly, hwidth=(0.0,0.05), fill='blue')
    
def draw_points(d, p_list):
    for pt in p_list:
        d.draw(pt, hradius=0.15, hwidth=0.02,
                   fill='white', opacity=0.9)

def ring(p, q, i0, i1, polygon_center_src):
    # construct a new poly centered around the origin (pts_center_src could be anywhere)
    pts_base = construct_poly_vertices(p, q)
    # TODO(lucasw) not sure what this does, why it only uses the first two points
    t0 = pts_base[0]
    t1 = pts_base[1]
    trans_to_origin = Transform.shift_origin(t0, t1)
    # print(f"trans_to_origin: {t0} {t1} -> {trans_to_origin}")
    if False:
        px, py = trans_to_origin.apply_to_tuple(pts_base[-1])
         # print('Inner angle:', math.degrees(math.atan2(py,px)))

    polygons = []
    # TODO(lucasw) need a datastructure that has a set of nodes with five neighbors each
    # and be able to track if a node has already been constructed in a given position
    
    # iterate through each edge of the polygon, and add a neighbor polygon
    for i in range(i0, i1):
        t0 = polygon_center_src.vertices[(i + 1) % p]
        t1 = polygon_center_src.vertices[i]
        # this seems like it would only be half the needed translation
        # it's the location of one edge of the center source polygon
        trans_to_side = Transform.translation(t0, t1)
        # print(f"{i} t: {tx} {ty} -> {trans_to_side}")
        transform = Transform.merge(trans_to_origin, trans_to_side)
        transformed_points = transform.apply_to_list(pts_base)
        transformed_polygon = Polygon.from_vertices(transformed_points)
        polygons.append(transformed_polygon)
   
    for polygon in polygons:
        draw_poly(d, polygon)
        
    # draw_points(d, p_list)
    # for pt_list2 in pt_list_list:
    #     draw_points(d, pt_list2)

    return polygons
    
# for ind in range(50):
ind = 25
if True:
    d = Drawing(2.1, 2.1, origin='center')
    d.draw(euclid.Circle(0, 0, 1), fill='#f9f9f9')

    p = 5
    q = 4
    # print('Inner angle:', 360/q)

    # the central polygon
    pt_list = construct_poly_vertices(p, q)
    rot_trans = Transform.rotation(deg=270)
    # offset = Transform.translation(Point(-0.5 + ind * 0.025, -0.0))
    # offset = Transform.translation(Point(-0.0, -0.5 + ind * 0.025))
    offset = Transform.translation(Point(0.25, 0.0))
    trans = Transform.merge(offset, rot_trans)
    pt_list = trans.apply_to_list(pt_list)
    poly = Polygon.from_vertices(pt_list)
    draw_poly(d, poly)

    polygons = ring(p, q, i0=0, i1=p, polygon_center_src=poly)

    # TODO(lucasw) need to avoid drawing the same shape twice
    if False:
        for polygon in polygons:
            polygons = ring(p, q, 1, 4, polygon)
            for polygon in polygons:
                polygons = ring(p, q, 0, 4, polygon)
                for polygon in polygons:
                    polygons = ring(p, q, 0, 5, polygon)
                    for polygon in polygons:
                        polygons = ring(p, q, 0, 5, polygon)

    # d.set_render_size(w=1080)
    d.set_render_size(w=400)
    name = f'images/poly_tile_{p}_{q}_{ind:05d}'
    d.save_png(name + ".png")
    # d.save_svg(name + ".svg")
d

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# print(pt_list)

xs, ys = get_poly_xy(poly)

fig, ax = plt.subplots()

for pt in pt_list:
    ax.plot(pt.x, pt.y, '-.')
    
# print(xs)
# print(ys)
    
ax.plot(xs, ys, '.')
ax.axis('equal')

In [ ]:
print(poly.vertices)

fig, ax = plt.subplots()
    
for poly in polygons:
    xs = []
    ys = []
    for vt in poly.vertices:
        xs.append(vt.x)
        ys.append(vt.y)
        
    xs.append(xs[0])
    ys.append(ys[0])
    
    ax.plot(xs, ys)
    ax.axis('equal')

In [ ]:
all_nodes = []

root_rot = Transform.rotation(deg=90)
root_offset = Transform.translation(Point(0.0, 0.0))
root_transform = Transform.merge(root_offset, root_rot)
root = Node(name="root", offset_transform=root_transform)
all_nodes.append(root)

children = []
children.extend(root.add_children("a"))
all_nodes.extend(children)
print(len(children))

grand_children = []
for i, child in enumerate(children):
    prefix = f"b{i}"
    grand_children.extend(child.add_children(prefix))
all_nodes.extend(grand_children)
print(len(grand_children))

if False:
    great_grand_children = []
    for i, child in enumerate(grand_children):
        prefix = f"c{i}"
        great_grand_children.extend(child.add_children(prefix))
    print(len(great_grand_children))
    all_nodes.extend(great_grand_children)
    
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

root.plot_recursive(ax)

ax.axis('equal')

fig, ax = plt.subplots()
for node in all_nodes:
    # poincare coordinates
    xp, yp = get_poly_xy(node.polygon)
    
    # instead of the unit circle poincare disk, convert back into
    # hyperboloid coordinates
    # zh^2 - xh^2 - yh^2 = 1.0
    # zh^2 = 1.0 + xh^2 + yh^2
    # zh = sqrt(1.0 + xh^2 + yh^2)
    
    # convert to hyperboloid coords, solve for scalar a which projects
    # the poincare point back onto the hyperboloid
    # (xh, yh, zh) = (0, 0, -1) + a * (xp, yp, 1.0) 
    # xh = a * xp
    # yh = a * yp
    # zh = a - 1.0
    # (a - 1.0)^2 = 1.0 + (a * xp)^2 + (a * yp)^2
    # a = 2.0 / (1 - xp^2 - yp^2)
    xp2 = np.multiply(xp, xp)
    yp2 = np.multiply(yp, yp)
    a = np.divide(2.0, 1.0 - xp2 - yp2)
    zh = a - 1.0
    xh = np.multiply(a, xp)
    yh = np.multiply(a, yp)
    
    # print zh * zh - 

    ax.plot(xh, yh)

ax.axis('equal')

if False:
    # TODO(lucasw) this isn't working
    drawing = Drawing(2.1, 2.1, origin='center')
    drawing.draw(euclid.Circle(0, 0, 1), fill='#f9f9f9')
    drawing.draw(root.polygon, fill="blue", opacity=1.0)
    print(drawing)
    drawing